In [2]:
import mido
from mido import Message, MidiFile, MidiTrack,MetaMessage
# from mido import MIDITime
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
# load data
label_pathe = r"D:\BrownUnivercity\CS2470\final_proj\motion_to_melody\data\EMOPIA_1.0\label.csv"
label = pd.read_csv(label_pathe)
# songs name ; emotion label ; who label

                    ID  4Q annotator
0     Q1_0vLPYiPN7qY_0   1         D
1     Q1_0vLPYiPN7qY_1   1         D
2     Q1_0vLPYiPN7qY_2   1         D
3     Q1_1Qc15G0ZHIg_1   1         D
4     Q1_1Qc15G0ZHIg_2   1         D
...                ...  ..       ...
1073  Q4_xgtwQGeB6_0_0   4         C
1074  Q4_xrhWli_R98g_0   4         C
1075  Q4_xrhWli_R98g_2   4         C
1076  Q4_yC0fAQxGgr0_0   4         C
1077  Q4_yC0fAQxGgr0_1   4         C

[1078 rows x 3 columns]


In [13]:
# Path to your MIDI file
midi_file_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\data\test\Q1_0vLPYiPN7qY_0.mid'

# Load the MIDI file
mid = mido.MidiFile(midi_file_path)

# Iterate over all messages in all tracks
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: {track.name}')
    for msg in track:
        print(msg)

Track 0: 
MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1)
Track 1: 
note_on channel=0 note=63 velocity=65 time=151
note_on channel=0 note=51 velocity=58 time=5
note_on channel=0 note=91 velocity=86 time=3
note_on channel=0 note=79 velocity=76 time=5
note_on channel=0 note=58 velocity=58 time=0
note_on channel=0 note=86 velocity=73 time=149
note_on channel=0 note=91 velocity=0 time=32
note_on channel=0 note=91 velocity=60 time=6
control_change channel=0 control=64 value=127 time=25
note_on channel=0 note=86 velocity=0 time=31
note_on channel=0 note=86 velocity=76 time=4
note_on channel=0 note=79 velocity=0 time=57
note_on channel=0 note=79 velocity=72 time=6
note_on channel=0 note=58 velocity=0 time=148
note_on channel=0 note=74 velocity=82 time=6
note_on channel=0 note=58 velocity=60 time=5
note_on channel=0 note=67 velocity=54 time=0


In [3]:

def get_song_msg(mid):
    one_song_msg = []
    for msg in mid.tracks[1]:
        if msg.type == 'note_on':
            note = msg.note
            velocity = msg.velocity
            time = msg.time
            temp = [note,velocity,time]
            one_song_msg.append(temp)
    return one_song_msg

    # print(len(one_song_msg))




In [44]:
# get emotion label
def get_emo(label, target):
    for i in range(len(label)):
        if label.iloc[i,0] == target:
            return label.iloc[i,1]


def get_label(label_path):
    label = pd.read_csv(label_path, delimiter='\t')
    # seperate the first column
    label[['name', '4Q', 'annotator']] = label['ID,4Q,annotator'].str.split(',', expand=True)
    # drop the original column
    label.drop(columns=['ID,4Q,annotator'], inplace=True) 
    return label

In [45]:

# label path     
label_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\data\label.csv'
# music folder path
folder_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\data\test'

# def get_music_data(folder_path, label_path):
#     music_data = []
#     label_data = []
#     for file_name in os.listdir(folder_path):
#         # Check if the file is a MIDI file
#         if file_name.endswith('.mid'):
#             # Construct the full path to the MIDI file
#             midi_file_path = os.path.join(folder_path, file_name)
            
#             # Load the MIDI file
#             mid = MidiFile(midi_file_path)
#             song_msg = get_song_msg(mid)

#         music_data.append(song_msg)
#         # label
#         filename_without_extension = os.path.splitext(file_name)[0]
#         labels = get_label(label_path)
#         # print(labels)
#         emo = get_emo(labels, filename_without_extension )
#         label_data.append(emo)

#     return music_data,label_data

def get_music_data(folder_path, label_path):
    music_data = []
    label_data = []
    align_length = 500
    for file_name in os.listdir(folder_path):
        # Check if the file is a MIDI file
        if file_name.endswith('.mid'):
            # Construct the full path to the MIDI file
            midi_file_path = os.path.join(folder_path, file_name)
            # Load the MIDI file
            mid = MidiFile(midi_file_path)
            song_msg = get_song_msg(mid)

        if len(song_msg) > align_length:
            result_array = song_msg[:align_length]
        else:
            num_to_pad = align_length - len(song_msg)
            result_array = np.pad(song_msg, ((0, num_to_pad), (0, 0)), mode='constant') 

        music_data.append(result_array)
        # label
        filename_without_extension = os.path.splitext(file_name)[0]
        labels = get_label(label_path)
        emo = get_emo(labels, filename_without_extension )
        label_data.append(emo)

    return music_data,label_data
    



In [46]:
music,label = get_music_data(folder_path, label_path)

In [47]:
# aab = []
# for i in range(len(music)):
#     aab.append(len(music[i]))

# plt.hist(aab)



In [48]:
print(music[0])

[[ 63  65 151]
 [ 51  58   5]
 [ 91  86   3]
 ...
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]]


In [11]:
def get_midi(events, output_file):
    mid = mido.MidiFile()
    track = mido.MidiTrack()

    # Set the tempo to default 120 BPM
    track.append(mido.MetaMessage('set_tempo', tempo=500000))

    # Iterate through the events and convert them to MIDI messages
    for event in events:
        note, velocity, time = event
        # Create a note_on message
        note_on = mido.Message('note_on', note=note, velocity=velocity, time=time)
        track.append(note_on)

    # Add the track to the MIDI file
    mid.tracks.append(track)

    # Save the MIDI file
    mid.save(output_file)

# MIDI events / list format
events = [[63, 65, 151], [51, 58, 5], [91, 86, 3], [79, 76, 5], [58, 58, 0],
           [86, 73, 149], [91, 0, 32], [91, 60, 6], [86, 0, 31], [86, 76, 4], [79, 0, 57],
             [79, 72, 6], [58, 0, 148], [74, 82, 6], [58, 60, 5], [67, 54, 0], [67, 0, 158], 
             [67, 75, 4], [79, 0, 141], [79, 87, 10]]

# Output MIDI file name
output_file = "output_1.mid"

# Reconstruct MIDI and save to file
get_midi(events, output_file)


In [25]:
# Path to your MIDI file
midi_file_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\data\test\Q1_0vLPYiPN7qY_0.mid'

# Load the MIDI file
mid = mido.MidiFile(midi_file_path)

# Iterate over all messages in all tracks
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: {track.name}')
    for msg in track:
        print(msg)
        
song_list =  get_song_msg(mid)


Track 0: 
MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1)
Track 1: 
note_on channel=0 note=63 velocity=65 time=151
note_on channel=0 note=51 velocity=58 time=5
note_on channel=0 note=91 velocity=86 time=3
note_on channel=0 note=79 velocity=76 time=5
note_on channel=0 note=58 velocity=58 time=0
note_on channel=0 note=86 velocity=73 time=149
note_on channel=0 note=91 velocity=0 time=32
note_on channel=0 note=91 velocity=60 time=6
control_change channel=0 control=64 value=127 time=25
note_on channel=0 note=86 velocity=0 time=31
note_on channel=0 note=86 velocity=76 time=4
note_on channel=0 note=79 velocity=0 time=57
note_on channel=0 note=79 velocity=72 time=6
note_on channel=0 note=58 velocity=0 time=148
note_on channel=0 note=74 velocity=82 time=6
note_on channel=0 note=58 velocity=60 time=5
note_on channel=0 note=67 velocity=54 time=0


In [26]:
output_file = "output_2.mid"
get_midi(song_list[:200], output_file)

In [28]:
song_list[40:80]

[[55, 0, 118],
 [55, 81, 8],
 [36, 54, 2],
 [52, 61, 2],
 [43, 53, 0],
 [64, 64, 2],
 [53, 0, 52],
 [74, 0, 3],
 [58, 0, 0],
 [86, 0, 0],
 [91, 0, 0],
 [67, 0, 1],
 [51, 0, 1],
 [62, 0, 1],
 [63, 0, 5],
 [46, 0, 8],
 [79, 0, 0],
 [60, 61, 77],
 [52, 0, 169],
 [55, 0, 8],
 [52, 44, 2],
 [55, 37, 3],
 [55, 0, 164],
 [55, 43, 4],
 [55, 0, 141],
 [60, 0, 0],
 [67, 56, 6],
 [55, 50, 0],
 [76, 68, 1],
 [60, 57, 1],
 [60, 0, 154],
 [67, 0, 0],
 [76, 0, 0],
 [76, 95, 6],
 [60, 76, 3],
 [67, 83, 1],
 [36, 0, 13],
 [43, 0, 12],
 [62, 61, 93],
 [77, 71, 12]]

In [24]:
# Path to your MIDI file
midi_file_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\code\demo\output_2.mid'

# Load the MIDI file
mid = mido.MidiFile(midi_file_path)

# Iterate over all messages in all tracks
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: {track.name}')
    for msg in track:
        print(msg)

Track 0: 
MetaMessage('set_tempo', tempo=500000, time=0)
note_on channel=0 note=63 velocity=65 time=151
note_on channel=0 note=51 velocity=58 time=5
note_on channel=0 note=91 velocity=86 time=3
note_on channel=0 note=79 velocity=76 time=5
note_on channel=0 note=58 velocity=58 time=0
note_on channel=0 note=86 velocity=73 time=149
note_on channel=0 note=91 velocity=0 time=32
note_on channel=0 note=91 velocity=60 time=6
note_on channel=0 note=86 velocity=0 time=31
note_on channel=0 note=86 velocity=76 time=4
note_on channel=0 note=79 velocity=0 time=57
note_on channel=0 note=79 velocity=72 time=6
note_on channel=0 note=58 velocity=0 time=148
note_on channel=0 note=74 velocity=82 time=6
note_on channel=0 note=58 velocity=60 time=5
note_on channel=0 note=67 velocity=54 time=0
note_on channel=0 note=67 velocity=0 time=158
note_on channel=0 note=67 velocity=75 time=4
note_on channel=0 note=79 velocity=0 time=141
note_on channel=0 note=79 velocity=87 time=10
MetaMessage('end_of_track', time=0)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [12]:
data = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]])

# Separate the data into the desired pattern
data_1 = data[:, ::3]   # Select every third element starting from the first
data_2 = data[:, 1::3]  # Select every third element starting from the second
data_3 = data[:, 2::3]  # Select every third element starting from the third

# Concatenate the separated data along the second dimension to get the desired result
separated_data = torch.cat((data_1, data_2, data_3), dim=1)

print(data_1)
print(data_2)
print(data_3)
print(separated_data)


tensor([[ 1,  4,  7, 10, 13]])
tensor([[ 2,  5,  8, 11, 14]])
tensor([[ 3,  6,  9, 12, 15]])
tensor([[ 1,  4,  7, 10, 13,  2,  5,  8, 11, 14,  3,  6,  9, 12, 15]])


In [18]:
# Given tensors
tensor1 = torch.tensor([[1, 4, 7, 10]])
tensor2 = torch.tensor([[2, 5, 8, 11]])
tensor3 = torch.tensor([[3, 6, 9, 12]])

# Stack the tensors
stacked_tensor = torch.stack((tensor1, tensor2, tensor3), dim=1)
stacked_tensor = stacked_tensor.squeeze()
print(stacked_tensor)
# Reshape to interleave elements
desired_order = stacked_tensor.t().flatten()
print("Interleaved Tensor:", desired_order)


tensor([[ 1,  4,  7, 10],
        [ 2,  5,  8, 11],
        [ 3,  6,  9, 12]])
Interleaved Tensor: tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])


In [3]:
def convert_reco(reconstruction):
    notes = reconstruction[::3] 
    velocity = reconstruction[1::3]  
    time = reconstruction[2::3]  
    return notes,velocity,time

def comb_reco(notes,velocity,time):
    stacked_tensor = torch.stack((notes, velocity, time), dim=1)
    reconstruction = stacked_tensor.reshape(-1)
    return reconstruction

tensor([1, 2, 3, 4])


In [9]:
data = [[1,2,3],[4,5,6],[7,8,9]]
data = torch.tensor(data)
data = data.view(-1, 9).to(torch.float32)
print(data)

tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9.]])


In [4]:

def min_max_normalize(tensor, min_value, max_value):
    min_val = tensor.min()
    max_val = tensor.max()
    normalized_tensor = min_value + (tensor - min_val) * (max_value - min_value) / (max_val - min_val)
    return normalized_tensor

velocity = torch.tensor([190,-250,-15, 1, 200, 23, 78], dtype=torch.float32)
normalized_velocity = min_max_normalize(velocity, 21, 108)

print(normalized_velocity)



tensor([106.0667,  21.0000,  66.4333,  69.5267, 108.0000,  73.7800,  84.4133])


In [ ]:
data = [25,78,91,23,45,67,60,-12,78,-43]
# random extract half of the data elements
# random 